<a href="https://colab.research.google.com/github/Abyanfl/Hck26/blob/main/P0LC2_abyan_naufal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

=================================================
Live Code 2

Nama  : Abyan Naufal
Batch : Hck-026

 program dibuat untuk melakukan analisa yang berkaitan dengan riwayat penjualan ke toko-toko yang berada di sekitar wilayah perusahaan.
=================================================



Pada tugas kali ini, dataset yang akan Anda gunakan berasal dari Google BigQuery. Gunakan informasi dibawah ini sebagai tempat untuk mengambil dataset.

    Project ID : ftds-hacktiv8-project
    Dataset : phase0_ftds_<nomor-batch>_<kode-lokasi-program>
    Tabel : beverage_sales

Contoh :

    Student offline : ftds-hacktiv8-project.phase0_ftds_001_hck.beverage_sales

    Sesuaikan dengan <nomor-batch> dan <kode-lokasi-program> Anda.

    Offline / Pondok Indah 	hck



In [1]:
from google.colab import auth
from google.cloud import bigquery
auth.authenticate_user()
print('Authenticated')

project_id_akun = "abyanlc2" # Gunakan GCP Project-ID Anda
client = bigquery.Client(project=project_id_akun)

Authenticated


In [19]:
import pandas as pd

In [26]:
query = '''
SELECT
*
FROM `ftds-hacktiv8-project.phase0_ftds_026_hck.beverage_sales`
'''

df_result = client.query(query).to_dataframe()
df_result.head()

,invoice_id,date,store_id,store_name,store_city,product_name,manufacturing_company,pack,unit_volume_ml,unit_sold,unit_retail_price,total_price,volume_sold_liters
0,INV-24279900015,2019-12-31,4119,Peak Provisions,CEDAR FALLS,MorningMist Dew,SipSavor Co.,12,1000,12,1.22,7.56,12.0
1,INV-19073300008,2019-04-30,4119,Peak Provisions,CEDAR FALLS,MorningMist Dew,SipSavor Co.,12,1000,12,1.22,7.56,12.0
2,INV-45354900010,2022-07-28,4119,Peak Provisions,CEDAR FALLS,MorningMist Dew,SipSavor Co.,12,1000,12,1.22,7.56,12.0
3,INV-39513900005,2021-08-24,4119,Peak Provisions,CEDAR FALLS,MorningMist Dew,SipSavor Co.,12,1000,12,1.22,7.56,12.0
4,INV-53810400018,2022-12-08,4119,Peak Provisions,CEDAR FALLS,MorningMist Dew,SipSavor Co.,12,1000,12,1.22,7.56,12.0


Section 1 - SQL

Jawablah masing-masing pertanyaan dibawah ini dengan menggunakan 1 query SQL per masing-masing pertanyaan. Anda wajib melakukan query SQL dengan menggunakan Google Colab (lihat bagian Dataset nomor 5 diatas sebagai contoh) :

1.Sebutkan kota yang masuk ke dalam wilayah distribusi penjualan perusahaan ?

In [27]:
query = '''
SELECT DISTINCT store_city
FROM `ftds-hacktiv8-project.phase0_ftds_026_hck.beverage_sales`
'''
df_result = client.query(query).to_dataframe()

df_result

,store_city
0,CEDAR FALLS
1,MASON CITY
2,CLINTON
3,CLARINDA
4,MUSCATINE
5,WEST DES MOINES


Berdasarkan hasil query diatas, perusahaan mendistribusikan produk nya ke 6 kota

Berapa rentang harga minimum dan maksimum dari unit minuman yang dijual ?

In [37]:
query = '''
SELECT
MIN(unit_retail_price) AS min_retail_price,
MAX(unit_retail_price) AS max_retail_price,
AVG(unit_retail_price) AS avg_retail_price

FROM `ftds-hacktiv8-project.phase0_ftds_026_hck.beverage_sales`
'''
df_result = client.query(query).to_dataframe()

df_result


,min_retail_price,max_retail_price,avg_retail_price
0,0.98,54.0,3.258816


In [77]:
query = '''
SELECT
MIN(total_price) AS min_price,
MAX(total_price) AS max_price,
AVG(total_price) AS avg_price

FROM `ftds-hacktiv8-project.phase0_ftds_026_hck.beverage_sales`
'''
df_result = client.query(query).to_dataframe()

df_result

,min_price,max_price,avg_price
0,-629.16,9828.0,54.916255


Bandingkan total unit yang terjual dan jumlah transaksi dari masing-masing kota pada tahun 2022 ?

In [85]:
query = '''
SELECT store_city,
COUNT(DISTINCT invoice_id) AS total_transaksi
SUM(unit_sold) AS total_unit_terjual
FROM `ftds-hacktiv8-project.phase0_ftds_026_hck.beverage_sales`
WHERE YEAR(date) = 2022
GROUP BY store_city
ORDER BY total_unit_terjual DESC;
'''
df_result = client.query(query).to_dataframe()

df_result



BadRequest: 400 Syntax error: Expected end of input but got identifier "SUM" at [4:1]; reason: invalidQuery, location: query, message: Syntax error: Expected end of input but got identifier "SUM" at [4:1]

Location: US
Job ID: 0b6e414b-312f-40f0-b922-bb3a5b216c16


Produk apa yang paling banyak diminati pada tahun 2022 ?

COUNT(*) AS total_sold → Counts best seller product

WHERE EXTRACT(YEAR FROM start_time) = 2022 → Filters for the year 2022

ORDER BY total_sold DESC → Sorts product from most to least.

LIMIT 1 → Returns only the most popular

In [43]:
query = '''
SELECT
    product_name,
    COUNT(*) AS total_sold
FROM `ftds-hacktiv8-project.phase0_ftds_026_hck.beverage_sales`
WHERE EXTRACT(YEAR FROM date) = 2022
GROUP BY product_name
ORDER BY total_sold DESC
LIMIT 1;

'''
df_result = client.query(query).to_dataframe()

df_result


,product_name,total_sold
0,FruitFusion Quencher,291


Section 2 - Pandas

Jawablah masing-masing pertanyaan dibawah ini dengan menggunakan Pandas.

Buatlah sebuah column baru dengan nama season. Column ini berisikan label yang menunjukkan keterangan musim berdasarkan bulan dengan ketentuan sebagai berikut : Bulan 	season
  Maret, April, Mei 	spring
  Juni, Juli, Agustus 	summer
  September, Oktober, November 	autumn
  Desember, Januari, Februari 	winter

    Petunjuk :

    - Untuk melakukan ekstraksi bulan, Anda dapat menggunakan Pandas.Series.dt.month. Pastikan bahwa column yang akan Anda extract waktu bulannya sudah dalam format datetime.

    - Setelah melakukan ekstraksi bulan, lakukan looping dan conditional untuk setiap entry data sesuai ketentuan kondisi keterangan musim diatas.

    - Simpan hasil looping dan conditional ini ke dalam sebuah list. Lakukan konversi dari list ke dalam sebuah column baru.

    Berapa total barang yang terjual kepada toko Peak Provisions sepanjang tahun 2022 ? (Gunakan pandas query untuk menjawab soal ini)


In [51]:
from google.colab import auth
from google.cloud import bigquery
import pandas as pd

# Authenticate
auth.authenticate_user()

# Connect to BigQuery
project_id = "abyanlc2"  # GUNAKAN GCP PROJECT-ID KALIAN MASING-MASING
client = bigquery.Client(project=project_id)

#query to retrieve all data
query = '''
SELECT
*
FROM `ftds-hacktiv8-project.phase0_ftds_026_hck.beverage_sales`
'''
#Load data into pandas dataframe
df = client.query(query).to_dataframe()

In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6014 entries, 0 to 6013
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   invoice_id             6014 non-null   object 
 1   date                   6014 non-null   dbdate 
 2   store_id               6014 non-null   Int64  
 3   store_name             6014 non-null   object 
 4   store_city             6014 non-null   object 
 5   product_name           6014 non-null   object 
 6   manufacturing_company  6014 non-null   object 
 7   pack                   6014 non-null   Int64  
 8   unit_volume_ml         6014 non-null   Int64  
 9   unit_sold              6014 non-null   Int64  
 10  unit_retail_price      6014 non-null   float64
 11  total_price            6014 non-null   float64
 12  volume_sold_liters     6014 non-null   float64
dtypes: Int64(4), dbdate(1), float64(3), object(5)
memory usage: 634.4+ KB


Create new Column name Season

In [75]:
# Pastikan 'date' dalam format datetime
df['date'] = pd.to_datetime(df['date'])

#ekstrak bulan
months =df['date'].dt.month

#menyimpan season
season_list = []

#looping untuk menentukan season
for month in months:
    if month in [3, 4, 5]:
      season_list.append('spring')
    elif month in [6, 7, 8]:
      season_list.append('summer')
    elif month in [9, 10, 11]:
      season_list.append('autumn')
    elif month in [12, 1, 2]:
      season_list.append('winter')
    else:
      season_list.append(none)

#tambah kolom season ke data frame
df['season'] = season_list

#tampilan hasil
df[['date', 'season']].head()


,date,season
0,2019-12-31,winter
1,2019-04-30,spring
2,2022-07-28,summer
3,2021-08-24,summer
4,2022-12-08,winter


Berapa total barang yang terjual kepada toko Peak Provisions sepanjang tahun 2022 ? (Gunakan pandas query untuk menjawab soal ini)